In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

---

## The Optimization Hierarchy

```
┌─────────────────────────────────────────────────────────────┐
│              OPTIMIZATION PRIORITY ORDER                    │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. ALGORITHM CHOICE          ███████████████████ ~10-100x  │
│     • Choose right algorithm                                │
│     • Reduce algorithmic complexity                         │
│                                                             │
│  2. MEMORY ACCESS PATTERNS    ████████████████    ~5-20x    │
│     • Coalesced access                                      │
│     • Minimize global memory traffic                        │
│     • Use shared memory                                     │
│                                                             │
│  3. OCCUPANCY & PARALLELISM   ████████████        ~2-5x     │
│     • Enough threads to hide latency                        │
│     • Balance resources per block                           │
│                                                             │
│  4. INSTRUCTION OPTIMIZATION  ████████            ~1.5-3x   │
│     • Fast math functions                                   │
│     • Avoid divergence                                      │
│                                                             │
│  5. CONCURRENCY               ██████              ~1.2-2x   │
│     • Streams, graphs                                       │
│     • Overlap compute/transfer                              │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## Part 1: Memory Optimization Techniques

### 1.1 Coalesced Memory Access

```cpp
// GOOD: Coalesced - threads access consecutive memory
__global__ void coalesced(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = data[tid] * 2.0f;  // Thread i accesses element i
    }
}

// BAD: Strided - threads access with stride
__global__ void strided(float* data, int n, int stride) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int idx = tid * stride;  // Thread i accesses element i*stride
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}
```

### 1.2 Shared Memory Usage

```cpp
// Pattern: Load to shared -> Sync -> Process -> Sync -> Store
__global__ void withShared(float* out, float* in, int n) {
    __shared__ float smem[256];
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;
    
    // Load to shared memory
    smem[tid] = (gid < n) ? in[gid] : 0.0f;
    __syncthreads();
    
    // Process using shared memory
    float result = smem[tid];
    if (tid > 0) result += smem[tid - 1];
    if (tid < 255) result += smem[tid + 1];
    __syncthreads();
    
    // Write result
    if (gid < n) out[gid] = result;
}
```

### 1.3 Bank Conflict Avoidance

```cpp
// BAD: Bank conflicts (stride of 32)
__shared__ float smem[32][32];
smem[threadIdx.x][0] = value;  // All access bank 0!

// GOOD: Padding to avoid conflicts
__shared__ float smem[32][33];  // +1 padding
smem[threadIdx.x][0] = value;   // Different banks
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile memory_patterns.cu
// memory_patterns.cu - Memory optimization examples
#include <stdio.h>
#include <cuda_runtime.h>

// GOOD: Coalesced - threads access consecutive memory
__global__ void coalesced(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = data[tid] * 2.0f;  // Thread i accesses element i
    }
}

// BAD: Strided - threads access with stride
__global__ void strided(float* data, int n, int stride) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int idx = tid * stride;  // Thread i accesses element i*stride
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

// Pattern: Load to shared -> Sync -> Process -> Sync -> Store
__global__ void withShared(float* out, float* in, int n) {
    __shared__ float smem[256];
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;
    
    // Load to shared memory
    smem[tid] = (gid < n) ? in[gid] : 0.0f;
    __syncthreads();
    
    // Process using shared memory
    float result = smem[tid];
    if (tid > 0) result += smem[tid - 1];
    if (tid < 255) result += smem[tid + 1];
    __syncthreads();
    
    // Write result
    if (gid < n) out[gid] = result;
}

int main() {
    const int N = 1 << 20;
    float *d_data, *d_out;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMalloc(&d_out, N * sizeof(float));
    
    // Initialize
    float* h_data = new float[N];
    for (int i = 0; i < N; i++) h_data[i] = 1.0f;
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    
    // Test coalesced
    coalesced<<<numBlocks, blockSize>>>(d_data, N);
    cudaDeviceSynchronize();
    printf("Coalesced access completed\n");
    
    // Test with shared memory
    withShared<<<numBlocks, blockSize>>>(d_out, d_data, N);
    cudaDeviceSynchronize();
    printf("Shared memory pattern completed\n");
    
    cudaFree(d_data);
    cudaFree(d_out);
    delete[] h_data;
    
    printf("Memory patterns demo complete!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o memory_patterns memory_patterns.cu
!./memory_patterns

---

## Part 2: Compute Optimization

### 2.1 Instruction Throughput

```cpp
// Fast math intrinsics (less accurate, much faster)
__device__ float fast_sin(float x) {
    return __sinf(x);      // ~10x faster than sinf()
}

__device__ float fast_exp(float x) {
    return __expf(x);      // ~10x faster than expf()
}

__device__ float fast_rsqrt(float x) {
    return rsqrtf(x);      // 1/sqrt(x), very fast
}
```

### 2.2 Warp Divergence

```cpp
// BAD: Divergent branches
__global__ void divergent(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid % 2 == 0) {      // Half warp does one thing
        data[tid] = expensive_op1(data[tid]);
    } else {                 // Other half does another
        data[tid] = expensive_op2(data[tid]);
    }
}

// BETTER: Separate into different warps
__global__ void nondivergent(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int warpId = tid / 32;
    
    if (warpId % 2 == 0) {   // Whole warp does one thing
        data[tid] = expensive_op1(data[tid]);
    } else {                 // Other warp does another
        data[tid] = expensive_op2(data[tid]);
    }
}
```

### 2.3 Loop Unrolling

```cpp
// Manual unroll
__global__ void unrolled(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    #pragma unroll 4
    for (int i = 0; i < 4; i++) {
        int idx = tid + i * blockDim.x * gridDim.x;
        if (idx < n) data[idx] *= 2.0f;
    }
}
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile compute_optimization.cu
// compute_optimization.cu - Compute optimization examples
#include <stdio.h>
#include <cuda_runtime.h>

// Fast math intrinsics (less accurate, much faster)
__device__ float fast_computation(float x) {
    float sin_val = __sinf(x);      // ~10x faster than sinf()
    float exp_val = __expf(x);      // ~10x faster than expf()
    float rsqrt_val = rsqrtf(x);    // 1/sqrt(x), very fast
    return sin_val + exp_val * rsqrt_val;
}

// BETTER: Non-divergent - whole warps take same path
__global__ void nondivergent(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int warpId = tid / 32;
    
    if (tid < n) {
        if (warpId % 2 == 0) {   // Whole warp does one thing
            data[tid] = __sinf(data[tid]);
        } else {                 // Other warp does another
            data[tid] = __cosf(data[tid]);
        }
    }
}

// Manual unroll
__global__ void unrolled(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    #pragma unroll 4
    for (int i = 0; i < 4; i++) {
        int idx = tid + i * blockDim.x * gridDim.x;
        if (idx < n) data[idx] *= 2.0f;
    }
}

int main() {
    const int N = 1 << 20;
    float *d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    // Initialize
    float* h_data = new float[N];
    for (int i = 0; i < N; i++) h_data[i] = 1.0f + (float)i / N;
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    
    // Test non-divergent
    nondivergent<<<numBlocks, blockSize>>>(d_data, N);
    cudaDeviceSynchronize();
    printf("Non-divergent kernel completed\n");
    
    // Test unrolled
    unrolled<<<numBlocks / 4, blockSize>>>(d_data, N);
    cudaDeviceSynchronize();
    printf("Unrolled kernel completed\n");
    
    cudaFree(d_data);
    delete[] h_data;
    
    printf("Compute optimization demo complete!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o compute_optimization compute_optimization.cu
!./compute_optimization

---

## Part 3: Occupancy Optimization

### 3.1 Resource Balancing

```cpp
// Query optimal block size
int minGridSize, optBlockSize;
cudaOccupancyMaxPotentialBlockSize(
    &minGridSize, &optBlockSize, 
    myKernel, 0, 0);

printf("Optimal block size: %d\n", optBlockSize);
printf("Min grid size: %d\n", minGridSize);
```

### 3.2 Register Pressure

```cpp
// Limit registers to increase occupancy
__global__ __launch_bounds__(256, 4)  // 256 threads, 4 blocks/SM
void limitedRegisters(float* data) {
    // Kernel code
}

// Compile with: nvcc -maxrregcount=32 kernel.cu
```

### 3.3 Shared Memory Configuration

```cpp
// Prefer more shared memory over L1 cache
cudaFuncSetCacheConfig(myKernel, cudaFuncCachePreferShared);

// Options:
// cudaFuncCachePreferNone   - No preference
// cudaFuncCachePreferShared - Prefer shared memory
// cudaFuncCachePreferL1     - Prefer L1 cache
// cudaFuncCachePreferEqual  - Equal split
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile occupancy_demo.cu
// occupancy_demo.cu - Occupancy optimization examples
#include <stdio.h>
#include <cuda_runtime.h>

// Limit registers to increase occupancy
__global__ __launch_bounds__(256, 4)  // 256 threads, 4 blocks/SM
void limitedRegisters(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = data[tid] * 2.0f;
    }
}

__global__ void myKernel(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = sqrtf(data[tid]) * 2.0f;
    }
}

int main() {
    const int N = 1 << 20;
    float *d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    // Query optimal block size
    int minGridSize, optBlockSize;
    cudaOccupancyMaxPotentialBlockSize(
        &minGridSize, &optBlockSize, 
        myKernel, 0, 0);
    
    printf("Optimal block size: %d\n", optBlockSize);
    printf("Min grid size: %d\n", minGridSize);
    
    // Calculate occupancy
    int numBlocks;
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(
        &numBlocks, myKernel, optBlockSize, 0);
    
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    
    int maxBlocksPerSM = prop.maxThreadsPerMultiProcessor / optBlockSize;
    float occupancy = (float)numBlocks / maxBlocksPerSM;
    printf("Max blocks per SM: %d\n", maxBlocksPerSM);
    printf("Achieved occupancy: %.1f%%\n", occupancy * 100);
    
    // Set cache preference
    cudaFuncSetCacheConfig(myKernel, cudaFuncCachePreferShared);
    printf("Set cache preference: PreferShared\n");
    
    // Launch kernel with optimal parameters
    int numBlocksTotal = (N + optBlockSize - 1) / optBlockSize;
    myKernel<<<numBlocksTotal, optBlockSize>>>(d_data, N);
    cudaDeviceSynchronize();
    
    printf("Kernel completed with optimal configuration!\n");
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o occupancy_demo occupancy_demo.cu
!./occupancy_demo

---

## Part 4: Concurrency Optimization

### 4.1 Stream Overlap Pattern

```cpp
// Chunk and overlap pattern
const int NUM_STREAMS = 4;
cudaStream_t streams[NUM_STREAMS];

for (int i = 0; i < NUM_STREAMS; i++) {
    cudaStreamCreate(&streams[i]);
}

int chunkSize = N / NUM_STREAMS;

for (int i = 0; i < NUM_STREAMS; i++) {
    int offset = i * chunkSize;
    
    // H2D for chunk i
    cudaMemcpyAsync(d_in + offset, h_in + offset,
                    chunkSize * sizeof(float),
                    cudaMemcpyHostToDevice, streams[i]);
    
    // Compute chunk i
    kernel<<<blocks, threads, 0, streams[i]>>>(
        d_out + offset, d_in + offset, chunkSize);
    
    // D2H for chunk i
    cudaMemcpyAsync(h_out + offset, d_out + offset,
                    chunkSize * sizeof(float),
                    cudaMemcpyDeviceToHost, streams[i]);
}
```

### 4.2 CUDA Graphs for Repeated Patterns

```cpp
// Capture repeating pattern
cudaGraph_t graph;
cudaGraphExec_t instance;

cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);

// Pattern to repeat
kernelA<<<...>>>(...);
kernelB<<<...>>>(...);
kernelC<<<...>>>(...);

cudaStreamEndCapture(stream, &graph);
cudaGraphInstantiate(&instance, graph, NULL, NULL, 0);

// Execute efficiently many times
for (int iter = 0; iter < 1000; iter++) {
    cudaGraphLaunch(instance, stream);
}
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile concurrency_demo.cu
// concurrency_demo.cu - Stream overlap and CUDA Graphs
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void kernel(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = sqrtf(data[tid]) * 2.0f;
    }
}

int main() {
    const int N = 1 << 22;  // 4M elements
    const int NUM_STREAMS = 4;
    size_t bytes = N * sizeof(float);
    int chunkSize = N / NUM_STREAMS;
    
    // Allocate pinned host memory
    float *h_in, *h_out;
    cudaMallocHost(&h_in, bytes);
    cudaMallocHost(&h_out, bytes);
    
    // Initialize
    for (int i = 0; i < N; i++) h_in[i] = (float)i;
    
    // Allocate device memory
    float *d_in, *d_out;
    cudaMalloc(&d_in, bytes);
    cudaMalloc(&d_out, bytes);
    
    // Create streams
    cudaStream_t streams[NUM_STREAMS];
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamCreate(&streams[i]);
    }
    
    int blockSize = 256;
    int blocks = (chunkSize + blockSize - 1) / blockSize;
    
    // ============================================
    // Stream Overlap Pattern
    // ============================================
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    
    for (int i = 0; i < NUM_STREAMS; i++) {
        int offset = i * chunkSize;
        
        // H2D for chunk i
        cudaMemcpyAsync(d_in + offset, h_in + offset,
                        chunkSize * sizeof(float),
                        cudaMemcpyHostToDevice, streams[i]);
        
        // Compute chunk i
        kernel<<<blocks, blockSize, 0, streams[i]>>>(
            d_in + offset, chunkSize);
        
        // D2H for chunk i
        cudaMemcpyAsync(h_out + offset, d_in + offset,
                        chunkSize * sizeof(float),
                        cudaMemcpyDeviceToHost, streams[i]);
    }
    
    // Sync all streams
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamSynchronize(streams[i]);
    }
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("Stream overlap: %.2f ms\n", ms);
    
    // ============================================
    // CUDA Graph for Repeated Pattern
    // ============================================
    cudaGraph_t graph;
    cudaGraphExec_t instance;
    
    // Reset data
    cudaMemcpy(d_in, h_in, bytes, cudaMemcpyHostToDevice);
    
    cudaStreamBeginCapture(streams[0], cudaStreamCaptureModeGlobal);
    
    // Pattern to repeat: 3 kernel passes
    kernel<<<(N + 255) / 256, 256, 0, streams[0]>>>(d_in, N);
    kernel<<<(N + 255) / 256, 256, 0, streams[0]>>>(d_in, N);
    kernel<<<(N + 255) / 256, 256, 0, streams[0]>>>(d_in, N);
    
    cudaStreamEndCapture(streams[0], &graph);
    cudaGraphInstantiate(&instance, graph, NULL, NULL, 0);
    
    cudaEventRecord(start);
    
    // Execute graph 10 times
    for (int iter = 0; iter < 10; iter++) {
        cudaGraphLaunch(instance, streams[0]);
    }
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    cudaEventElapsedTime(&ms, start, stop);
    printf("CUDA Graph (10 iterations x 3 kernels): %.2f ms\n", ms);
    
    // Cleanup
    cudaGraphExecDestroy(instance);
    cudaGraphDestroy(graph);
    
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamDestroy(streams[i]);
    }
    
    cudaFree(d_in);
    cudaFree(d_out);
    cudaFreeHost(h_in);
    cudaFreeHost(h_out);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    printf("Concurrency demo complete!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o concurrency_demo concurrency_demo.cu
!./concurrency_demo

---

## Optimization Checklist

```
┌─────────────────────────────────────────────────────────────┐
│              CUDA OPTIMIZATION CHECKLIST                    │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  □ MEMORY ACCESS                                            │
│    □ Coalesced global memory access                         │
│    □ Minimize global memory transactions                    │
│    □ Use shared memory for reused data                      │
│    □ Avoid bank conflicts in shared memory                  │
│    □ Use __ldg() for read-only data                         │
│    □ Use pinned memory for host-device transfers            │
│                                                             │
│  □ COMPUTE                                                  │
│    □ Use fast math where precision allows                   │
│    □ Minimize warp divergence                               │
│    □ Use appropriate data types (float vs double)           │
│    □ Unroll loops where beneficial                          │
│    □ Use warp-level primitives                              │
│                                                             │
│  □ OCCUPANCY                                                │
│    □ Use occupancy calculator for block size                │
│    □ Balance registers vs occupancy                         │
│    □ Balance shared memory vs occupancy                     │
│    □ Ensure enough blocks to saturate GPU                   │
│                                                             │
│  □ CONCURRENCY                                              │
│    □ Overlap compute and memory transfers                   │
│    □ Use multiple streams for independent work              │
│    □ Use CUDA Graphs for repeated patterns                  │
│    □ Enable peer access for multi-GPU                       │
│                                                             │
│  □ PROFILING                                                │
│    □ Profile with Nsight Compute                            │
│    □ Check roofline position                                │
│    □ Identify bottlenecks before optimizing                 │
│    □ Measure improvement after each change                  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## Complete Optimized Example: Matrix Transpose

### 🔷 CUDA C++ Implementation (Primary)

```cpp
// optimized_transpose.cu - Fully optimized matrix transpose
#include <stdio.h>
#include <cuda_runtime.h>

#define TILE_DIM 32
#define BLOCK_ROWS 8

// Naive transpose (for comparison)
__global__ void transposeNaive(float* out, float* in, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (x < width && y < height) {
        out[x * height + y] = in[y * width + x];
    }
}

// Optimized transpose with shared memory and bank conflict avoidance
__global__ void transposeOptimized(float* out, float* in, int width, int height) {
    // +1 padding to avoid bank conflicts
    __shared__ float tile[TILE_DIM][TILE_DIM + 1];
    
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    // Load tile (coalesced read)
    #pragma unroll
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < width && (y + j) < height) {
            tile[threadIdx.y + j][threadIdx.x] = in[(y + j) * width + x];
        }
    }
    
    __syncthreads();
    
    // Transposed coordinates
    x = blockIdx.y * TILE_DIM + threadIdx.x;
    y = blockIdx.x * TILE_DIM + threadIdx.y;
    
    // Store transposed tile (coalesced write)
    #pragma unroll
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < height && (y + j) < width) {
            out[(y + j) * height + x] = tile[threadIdx.x][threadIdx.y + j];
        }
    }
}

int main() {
    const int WIDTH = 4096, HEIGHT = 4096;
    size_t bytes = WIDTH * HEIGHT * sizeof(float);
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, bytes);
    cudaMalloc(&d_out, bytes);
    
    dim3 block(TILE_DIM, BLOCK_ROWS);
    dim3 grid((WIDTH + TILE_DIM - 1) / TILE_DIM,
              (HEIGHT + TILE_DIM - 1) / TILE_DIM);
    
    // Warmup
    transposeOptimized<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    
    // Benchmark
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    const int RUNS = 100;
    
    cudaEventRecord(start);
    for (int i = 0; i < RUNS; i++) {
        transposeOptimized<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    float bandwidth = 2.0f * bytes / (ms / RUNS / 1000.0f) / 1e9;
    printf("Optimized Transpose: %.2f GB/s\n", bandwidth);
    
    cudaFree(d_in);
    cudaFree(d_out);
    
    return 0;
}
```

In [ ]:
%%writefile optimized_transpose.cu
// optimized_transpose.cu - Fully optimized matrix transpose
#include <stdio.h>
#include <cuda_runtime.h>

#define TILE_DIM 32
#define BLOCK_ROWS 8

// Naive transpose (for comparison)
__global__ void transposeNaive(float* out, float* in, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (x < width && y < height) {
        out[x * height + y] = in[y * width + x];
    }
}

// Optimized transpose with shared memory and bank conflict avoidance
__global__ void transposeOptimized(float* out, float* in, int width, int height) {
    // +1 padding to avoid bank conflicts
    __shared__ float tile[TILE_DIM][TILE_DIM + 1];
    
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    // Load tile (coalesced read)
    #pragma unroll
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < width && (y + j) < height) {
            tile[threadIdx.y + j][threadIdx.x] = in[(y + j) * width + x];
        }
    }
    
    __syncthreads();
    
    // Transposed coordinates
    x = blockIdx.y * TILE_DIM + threadIdx.x;
    y = blockIdx.x * TILE_DIM + threadIdx.y;
    
    // Store transposed tile (coalesced write)
    #pragma unroll
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < height && (y + j) < width) {
            out[(y + j) * height + x] = tile[threadIdx.x][threadIdx.y + j];
        }
    }
}

int main() {
    const int WIDTH = 4096, HEIGHT = 4096;
    size_t bytes = WIDTH * HEIGHT * sizeof(float);
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, bytes);
    cudaMalloc(&d_out, bytes);
    
    // Initialize input
    float* h_in = new float[WIDTH * HEIGHT];
    for (int i = 0; i < WIDTH * HEIGHT; i++) h_in[i] = (float)i;
    cudaMemcpy(d_in, h_in, bytes, cudaMemcpyHostToDevice);
    
    dim3 block(TILE_DIM, BLOCK_ROWS);
    dim3 grid((WIDTH + TILE_DIM - 1) / TILE_DIM,
              (HEIGHT + TILE_DIM - 1) / TILE_DIM);
    
    // Warmup
    transposeOptimized<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    // Benchmark
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    const int RUNS = 100;
    float ms;
    
    // Benchmark naive
    cudaEventRecord(start);
    for (int i = 0; i < RUNS; i++) {
        transposeNaive<<<grid, dim3(32, 32)>>>(d_out, d_in, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms, start, stop);
    
    float naiveBW = 2.0f * bytes / (ms / RUNS / 1000.0f) / 1e9;
    printf("Naive Transpose:     %.2f GB/s\n", naiveBW);
    
    // Benchmark optimized
    cudaEventRecord(start);
    for (int i = 0; i < RUNS; i++) {
        transposeOptimized<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms, start, stop);
    
    float optBW = 2.0f * bytes / (ms / RUNS / 1000.0f) / 1e9;
    printf("Optimized Transpose: %.2f GB/s\n", optBW);
    printf("Speedup: %.2fx\n", optBW / naiveBW);
    
    cudaFree(d_in);
    cudaFree(d_out);
    delete[] h_in;
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 -o optimized_transpose optimized_transpose.cu
!./optimized_transpose

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to review and practice optimization techniques:

**Exercise 1: Memory Coalescing**
- Fix a kernel with poor memory access patterns
- Measure the performance improvement

**Exercise 2: Shared Memory Optimization**
- Convert a naive matrix operation to use shared memory
- Handle bank conflicts appropriately

**Exercise 3: Occupancy Analysis**
- Experiment with different block sizes
- Find the optimal configuration for your GPU

In [ ]:
%%writefile optimization_review_exercises.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error at %s:%d: %s\n", __FILE__, __LINE__, \
                   cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: Memory Coalescing
// The "bad" kernel has strided access; fix it in the "good" kernel
// =============================================================================

// BAD: Strided memory access (each thread reads columns)
__global__ void transposeNaive(float* out, float* in, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (x < width && y < height) {
        // Reading: row-major with stride (bad for reads when x varies slowly)
        // Writing: column-major (bad for writes)
        out[x * height + y] = in[y * width + x];
    }
}

// TODO: Exercise 1 - Fix the memory access pattern using shared memory
// Use a tile of shared memory to enable coalesced reads and writes
#define TILE_DIM 32
#define BLOCK_ROWS 8

__global__ void transposeOptimized(float* out, float* in, int width, int height) {
    __shared__ float tile[TILE_DIM][TILE_DIM + 1];  // +1 to avoid bank conflicts
    
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    // TODO: Exercise 1a - Load tile with coalesced reads
    // Each thread loads multiple elements (TILE_DIM / BLOCK_ROWS)
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < width && (y + j) < height) {
            tile[threadIdx.y + j][threadIdx.x] = in[(y + j) * width + x];
        }
    }
    
    __syncthreads();
    
    // TODO: Exercise 1b - Write tile with coalesced writes
    // Note: indices are swapped for transpose
    x = blockIdx.y * TILE_DIM + threadIdx.x;
    y = blockIdx.x * TILE_DIM + threadIdx.y;
    
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < height && (y + j) < width) {
            out[(y + j) * height + x] = tile[threadIdx.x][threadIdx.y + j];
        }
    }
}

// =============================================================================
// Exercise 2: Shared Memory - Matrix Row Sum
// =============================================================================

// Naive: Each thread reads entire row (lots of global memory traffic)
__global__ void rowSumNaive(float* out, float* matrix, int rows, int cols) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < rows) {
        float sum = 0.0f;
        for (int c = 0; c < cols; c++) {
            sum += matrix[row * cols + c];
        }
        out[row] = sum;
    }
}

// TODO: Exercise 2 - Use shared memory for collaborative loading
__global__ void rowSumOptimized(float* out, float* matrix, int rows, int cols) {
    extern __shared__ float sdata[];
    
    int row = blockIdx.x;
    int tid = threadIdx.x;
    
    // TODO: Exercise 2a - Each thread loads multiple columns collaboratively
    float sum = 0.0f;
    for (int c = tid; c < cols; c += blockDim.x) {
        sum += matrix[row * cols + c];
    }
    
    // TODO: Exercise 2b - Store partial sum to shared memory
    sdata[tid] = sum;
    __syncthreads();
    
    // TODO: Exercise 2c - Parallel reduction in shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    // Write result
    if (tid == 0) {
        out[row] = sdata[0];
    }
}

// =============================================================================
// Exercise 3: Occupancy Optimization
// =============================================================================

// Vector add with configurable block size
__global__ void vectorAdd(float* c, float* a, float* b, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

void testOccupancy() {
    printf("=== Exercise 3: Occupancy Analysis ===\n");
    
    int n = 10 * 1024 * 1024;  // 10M elements
    
    float *d_a, *d_b, *d_c;
    CHECK_CUDA(cudaMalloc(&d_a, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_c, n * sizeof(float)));
    
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaEventCreate(&start));
    CHECK_CUDA(cudaEventCreate(&stop));
    
    // Test different block sizes
    int blockSizes[] = {32, 64, 128, 256, 512, 1024};
    int numSizes = sizeof(blockSizes) / sizeof(blockSizes[0]);
    
    printf("\n%-12s %-12s %-12s %-15s\n", "Block Size", "Num Blocks", "Occupancy", "Time (ms)");
    printf("─────────────────────────────────────────────────────\n");
    
    for (int i = 0; i < numSizes; i++) {
        int blockSize = blockSizes[i];
        int numBlocks = (n + blockSize - 1) / blockSize;
        
        // Calculate theoretical occupancy
        int maxActiveBlocks;
        CHECK_CUDA(cudaOccupancyMaxActiveBlocksPerMultiprocessor(
            &maxActiveBlocks, vectorAdd, blockSize, 0));
        
        cudaDeviceProp prop;
        CHECK_CUDA(cudaGetDeviceProperties(&prop, 0));
        
        float occupancy = (float)(maxActiveBlocks * blockSize) / prop.maxThreadsPerMultiProcessor;
        
        // Warmup
        vectorAdd<<<numBlocks, blockSize>>>(d_c, d_a, d_b, n);
        CHECK_CUDA(cudaDeviceSynchronize());
        
        // Time it
        CHECK_CUDA(cudaEventRecord(start));
        for (int j = 0; j < 100; j++) {
            vectorAdd<<<numBlocks, blockSize>>>(d_c, d_a, d_b, n);
        }
        CHECK_CUDA(cudaEventRecord(stop));
        CHECK_CUDA(cudaEventSynchronize(stop));
        
        float ms;
        CHECK_CUDA(cudaEventElapsedTime(&ms, start, stop));
        
        printf("%-12d %-12d %-12.1f%% %-15.3f\n", 
               blockSize, numBlocks, occupancy * 100, ms / 100);
    }
    
    printf("\n");
    
    // Use occupancy calculator to find optimal
    int minGridSize, optBlockSize;
    CHECK_CUDA(cudaOccupancyMaxPotentialBlockSize(
        &minGridSize, &optBlockSize, vectorAdd, 0, 0));
    
    printf("Suggested optimal block size: %d\n", optBlockSize);
    printf("Suggested minimum grid size: %d\n\n", minGridSize);
    
    CHECK_CUDA(cudaEventDestroy(start));
    CHECK_CUDA(cudaEventDestroy(stop));
    CHECK_CUDA(cudaFree(d_a));
    CHECK_CUDA(cudaFree(d_b));
    CHECK_CUDA(cudaFree(d_c));
}

// Test Exercise 1
void testTranspose() {
    printf("=== Exercise 1: Memory Coalescing (Transpose) ===\n");
    
    int width = 4096, height = 4096;
    size_t size = width * height * sizeof(float);
    
    float *h_in = (float*)malloc(size);
    float *h_out = (float*)malloc(size);
    
    for (int i = 0; i < width * height; i++) h_in[i] = (float)i;
    
    float *d_in, *d_out;
    CHECK_CUDA(cudaMalloc(&d_in, size));
    CHECK_CUDA(cudaMalloc(&d_out, size));
    CHECK_CUDA(cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice));
    
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaEventCreate(&start));
    CHECK_CUDA(cudaEventCreate(&stop));
    
    dim3 block(TILE_DIM, BLOCK_ROWS);
    dim3 grid((width + TILE_DIM - 1) / TILE_DIM, (height + TILE_DIM - 1) / TILE_DIM);
    
    // Naive
    CHECK_CUDA(cudaEventRecord(start));
    for (int i = 0; i < 10; i++) {
        transposeNaive<<<grid, block>>>(d_out, d_in, width, height);
    }
    CHECK_CUDA(cudaEventRecord(stop));
    CHECK_CUDA(cudaEventSynchronize(stop));
    float naiveTime;
    CHECK_CUDA(cudaEventElapsedTime(&naiveTime, start, stop));
    
    // Optimized
    CHECK_CUDA(cudaEventRecord(start));
    for (int i = 0; i < 10; i++) {
        transposeOptimized<<<grid, block>>>(d_out, d_in, width, height);
    }
    CHECK_CUDA(cudaEventRecord(stop));
    CHECK_CUDA(cudaEventSynchronize(stop));
    float optTime;
    CHECK_CUDA(cudaEventElapsedTime(&optTime, start, stop));
    
    printf("Naive transpose: %.3f ms\n", naiveTime / 10);
    printf("Optimized transpose: %.3f ms\n", optTime / 10);
    printf("Speedup: %.2fx\n\n", naiveTime / optTime);
    
    CHECK_CUDA(cudaEventDestroy(start));
    CHECK_CUDA(cudaEventDestroy(stop));
    CHECK_CUDA(cudaFree(d_in));
    CHECK_CUDA(cudaFree(d_out));
    free(h_in);
    free(h_out);
}

// Test Exercise 2
void testRowSum() {
    printf("=== Exercise 2: Shared Memory (Row Sum) ===\n");
    
    int rows = 4096, cols = 4096;
    size_t matrixSize = rows * cols * sizeof(float);
    size_t outSize = rows * sizeof(float);
    
    float *h_matrix = (float*)malloc(matrixSize);
    float *h_out = (float*)malloc(outSize);
    
    for (int i = 0; i < rows * cols; i++) h_matrix[i] = 1.0f;
    
    float *d_matrix, *d_out;
    CHECK_CUDA(cudaMalloc(&d_matrix, matrixSize));
    CHECK_CUDA(cudaMalloc(&d_out, outSize));
    CHECK_CUDA(cudaMemcpy(d_matrix, h_matrix, matrixSize, cudaMemcpyHostToDevice));
    
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaEventCreate(&start));
    CHECK_CUDA(cudaEventCreate(&stop));
    
    // Naive
    CHECK_CUDA(cudaEventRecord(start));
    for (int i = 0; i < 10; i++) {
        rowSumNaive<<<(rows + 255) / 256, 256>>>(d_out, d_matrix, rows, cols);
    }
    CHECK_CUDA(cudaEventRecord(stop));
    CHECK_CUDA(cudaEventSynchronize(stop));
    float naiveTime;
    CHECK_CUDA(cudaEventElapsedTime(&naiveTime, start, stop));
    
    // Optimized
    CHECK_CUDA(cudaEventRecord(start));
    for (int i = 0; i < 10; i++) {
        rowSumOptimized<<<rows, 256, 256 * sizeof(float)>>>(d_out, d_matrix, rows, cols);
    }
    CHECK_CUDA(cudaEventRecord(stop));
    CHECK_CUDA(cudaEventSynchronize(stop));
    float optTime;
    CHECK_CUDA(cudaEventElapsedTime(&optTime, start, stop));
    
    CHECK_CUDA(cudaMemcpy(h_out, d_out, outSize, cudaMemcpyDeviceToHost));
    
    printf("Naive row sum: %.3f ms\n", naiveTime / 10);
    printf("Optimized row sum: %.3f ms\n", optTime / 10);
    printf("Speedup: %.2fx\n", naiveTime / optTime);
    printf("Sample result (expected %d): %.0f\n\n", cols, h_out[0]);
    
    CHECK_CUDA(cudaEventDestroy(start));
    CHECK_CUDA(cudaEventDestroy(stop));
    CHECK_CUDA(cudaFree(d_matrix));
    CHECK_CUDA(cudaFree(d_out));
    free(h_matrix);
    free(h_out);
}

int main() {
    printf("\n╔══════════════════════════════════════════════════════════════╗\n");
    printf("║           Optimization Review Exercises                      ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    testTranspose();
    testRowSum();
    testOccupancy();
    
    printf("═══════════════════════════════════════════════════════════════\n");
    printf("Study the optimized versions and try profiling with Nsight!\n");
    printf("═══════════════════════════════════════════════════════════════\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 -o optimization_review_exercises optimization_review_exercises.cu && ./optimization_review_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises practice optimization concepts using Numba:

```python
from numba import cuda
import numpy as np

# Exercise 1: Compare coalesced vs non-coalesced access
@cuda.jit
def coalesced_copy(out, inp):
    """Good: Coalesced access - consecutive threads access consecutive memory"""
    idx = cuda.grid(1)
    if idx < inp.size:
        out[idx] = inp[idx]

@cuda.jit
def strided_copy(out, inp, stride):
    """Bad: Strided access - threads access memory with gaps"""
    idx = cuda.grid(1)
    strided_idx = (idx * stride) % inp.size
    if idx < inp.size:
        out[idx] = inp[strided_idx]

# Exercise 2: Shared memory tile
@cuda.jit
def tiled_operation(out, inp):
    """Use shared memory for data reuse"""
    tile = cuda.shared.array(256, dtype=numba.float32)
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    idx = bx * cuda.blockDim.x + tx
    
    # Load to shared memory
    if idx < inp.size:
        tile[tx] = inp[idx]
    cuda.syncthreads()
    
    # Process with neighbors (data reuse from shared memory)
    # ...
```

**Tip**: Profile with `cuda.profiling()` context manager to measure kernel times.

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────────┐
│              OPTIMIZATION SUMMARY                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. Profile First                                           │
│     • Identify bottleneck before optimizing                 │
│     • Don't guess, measure                                  │
│                                                             │
│  2. Memory is Usually the Bottleneck                        │
│     • Coalescing is critical                                │
│     • Shared memory for data reuse                          │
│     • Minimize transfers                                    │
│                                                             │
│  3. Occupancy Matters (to a point)                          │
│     • Need enough parallelism to hide latency               │
│     • But higher isn't always better                        │
│                                                             │
│  4. Concurrency for Free Performance                        │
│     • Overlap compute and transfers                         │
│     • Use streams and graphs                                │
│                                                             │
│  5. Iterate and Measure                                     │
│     • One optimization at a time                            │
│     • Verify improvement after each change                  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

## Next: Day 4 - Capstone Project